# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [31]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [32]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,utrik,11.2278,169.8474,27.65,71,74,9.10,MH,1700749536
1,1,mwanza,-2.5167,32.9000,26.39,56,44,1.59,TZ,1700749536
2,2,isafjordur,66.0755,-23.1240,-1.98,64,40,3.60,IS,1700749536
3,3,albany,42.6001,-73.9662,5.76,77,49,0.89,US,1700749536
4,4,saint-amand-montrond,46.7500,2.5000,7.57,73,87,1.34,FR,1700749537


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [33]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "Humidity",
    ylabel = "Latitude",
    xlabel = "Longtitude",
    title  = "Plot to show the Humidity(%) for each City"
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [34]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_df = city_data_df.loc[(city_data_df["Max Temp"]<27) & (city_data_df["Max Temp"]>21.00) & (city_data_df["Wind Speed"]<4.5) & (city_data_df["Cloudiness"]==0)]

# Drop any rows with null values
ideal_city_df.dropna(how="any")

# Display sample data
ideal_city_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
8,8,margaret river,-33.9500,115.0667,23.52,39,0,4.43,AU,1700749537
108,108,ixtapa,20.7000,-105.2000,21.79,94,0,2.57,MX,1700749546
128,128,siwa oasis,29.2041,25.5195,23.73,25,0,0.83,EG,1700749548
168,168,haiku-pauwela,20.9219,-156.3051,24.47,86,0,0.00,US,1700749553
182,182,rocha,-34.4833,-54.3333,23.83,34,0,3.58,UY,1700749554


### Step 3: Create a new DataFrame called `hotel_df`.

In [38]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_df[["City","Country","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
8,margaret river,AU,-33.9500,115.0667,39,
108,ixtapa,MX,20.7000,-105.2000,94,
128,siwa oasis,EG,29.2041,25.5195,25,
168,haiku-pauwela,US,20.9219,-156.3051,86,
182,rocha,UY,-34.4833,-54.3333,34,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [45]:
# Set parameters to search for a hotel
radius = 10000
limit = 1

params = {
    "limit":limit,
    "apiKey":geoapify_key,
    "categories":"accommodation.hotel",
    "name":"en"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
margaret river - nearest hotel: No hotel found
ixtapa - nearest hotel: Hacienda Buenaventura Hotel & Mexican Charm
siwa oasis - nearest hotel: Taghaghien Island Resort
haiku-pauwela - nearest hotel: No hotel found
rocha - nearest hotel: No hotel found
ambagarh chauki - nearest hotel: No hotel found
carlet - nearest hotel: No hotel found
ghat - nearest hotel: No hotel found
howrah - nearest hotel: Golden Apple
al hufuf - nearest hotel: Al Muhaidab Residence Al Ahsa
veraval - nearest hotel: No hotel found
ha'il - nearest hotel: No hotel found
sini - nearest hotel: No hotel found
castillos - nearest hotel: A mi gente
cabo san lucas - nearest hotel: Seven Crown Hotel
iranshahr - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
8,margaret river,AU,-33.9500,115.0667,39,No hotel found
108,ixtapa,MX,20.7000,-105.2000,94,Hacienda Buenaventura Hotel & Mexican Charm
128,siwa oasis,EG,29.2041,25.5195,25,Taghaghien Island Resort
168,haiku-pauwela,US,20.9219,-156.3051,86,No hotel found
182,rocha,UY,-34.4833,-54.3333,34,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [46]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 2,
    color = "Humidity",
    ylabel = "Latitude",
    xlabel = "Longtitude",
    title  = "Plot to show the Humidity(%) for each City",
    hover_cols = ["Hotel Name","Country"]
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,Hotel Name,Country)